In [200]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA

In [492]:
data = pd.read_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\train.csv")
test = pd.read_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\test.csv")
data.shape

(1460, 81)

In [493]:
cat_cols = data.select_dtypes(exclude=[np.number])
num_cols = data.select_dtypes(include=[np.number])

In [494]:
cat_cols.describe()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
count,1460,1460,91,1460,1460,1460,1460,1460,1460,1460,...,1379,1379,1379,1379,1460,7,281,54,1460,1460
unique,5,2,2,4,4,2,5,3,25,9,...,6,3,5,5,3,3,4,4,9,6
top,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal
freq,1151,1454,50,925,1311,1459,1052,1382,225,1260,...,870,605,1311,1326,1340,3,157,49,1267,1198


In [495]:
for cols in cat_cols.columns:
    data[cols + "_code" ] = data[cols].astype('category').cat.codes

In [496]:
data = data.select_dtypes(include = [np.number]).interpolate().dropna()

In [497]:
cat = []
for cols in cat_cols.columns:
    cat.append(cols+"_code")
for cols in num_cols.columns:
    cat.append(cols)

In [498]:
train = data[cat]

In [499]:
sum(train.isnull().sum()!=0)

0

In [500]:
Y = np.log(train.SalePrice)
X = train.drop(['SalePrice','Id'],axis = 1)

In [501]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state = 44, test_size = .30)

In [502]:
lr = linear_model.LinearRegression()

In [503]:
model = lr.fit(X_train,Y_train)

In [504]:
model.score(X_test, Y_test)

0.8355365965375574

In [505]:
pred = model.predict(X_test)

In [506]:
mean_squared_error(Y_test, pred)

0.02661576206848082

Genrate predictions

In [507]:
for cols in cat_cols.columns:
    test[cols + "_code" ] = test[cols].astype('category').cat.codes

In [508]:
test = test.select_dtypes(include=[np.number]).interpolate().dropna()

In [509]:
cat.remove('SalePrice')

In [510]:
test_data = test[cat] 

In [511]:
submission = pd.DataFrame()
submission["Id"] = test_data.Id
test_data = test_data.drop(['Id'],axis = 1)

In [512]:
pred = model.predict(test_data)

In [513]:
pred[:6]

array([11.64017481, 11.96792516, 11.99959552, 12.14102077, 12.10999916,
       12.03638351])

In [514]:
pred = np.exp(pred)

In [515]:
submission['SalePrice'] = pred

In [516]:
submission.to_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\submission1.csv", index=False)

In [123]:
for i in range (-2, 4):
    alpha = 10**i
    print(alpha,"\n")
    rm = linear_model.Ridge(alpha=alpha)
    ridge_model = rm.fit(X_train, Y_train)
    preds_ridge = ridge_model.predict(X_test)
    print(mean_squared_error(Y_test,preds_ridge))

0.01 

0.0265939155743259
0.1 

0.026407385462780854
1 

0.025200426721766273
10 

0.023356314364559625
100 

0.02270194557790402
1000 

0.025127733454935623


In [120]:
alpha = 100
rm = linear_model.Ridge(alpha=alpha)
ridge_model = rm.fit(X_train, Y_train)
pred = ridge_model.predict(test_data)

In [121]:
pred[:5]

array([11.79112494, 11.96022984, 12.09243263, 12.02772144, 12.18862743])

In [122]:
pred = np.exp(pred)
submission['SalePrice'] = pred
submission.to_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\submission2.csv", index=False)

# PCA

In [127]:
numeric = num_cols.drop(["Id","SalePrice"], axis = 1).columns

In [137]:
pca_data = data[numeric]
pca_data = pca_data.interpolate().dropna()

In [138]:
sc = StandardScaler()
pca_data = sc.fit_transform(pca_data)

In [253]:
pca = PCA(n_components= 0.99)

In [254]:
pca95= pca.fit_transform(pca_data)

In [255]:
X_train,X_test,Y_train,Y_test = train_test_split(pca95,Y,random_state = 44, test_size = .30)

In [161]:
model = lr.fit(X_train,Y_train)

In [162]:
pred = model.predict(X_test)

In [163]:
mean_squared_error(Y_test,pred)

0.024264455654934777

In [261]:
test_data = test[numeric]
test_data = test_data.interpolate().dropna()
test_data = sc.fit_transform(test_data)

In [262]:
test_data = pca.fit_transform(test_data)
test_data.shape

(1459, 32)

In [172]:
pred = model.predict(test_data)

In [173]:
pred[:6]

array([11.66935162, 12.43913204, 11.94703526, 12.01389477, 12.12159802,
       11.99766378])

In [174]:
pred = np.exp(pred)
submission['SalePrice'] = pred
submission.to_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\submission3.csv", index=False)

Combine the categorical variables with the PCA data

In [217]:
cat_var = []
for col in cat_cols.columns:
    cat_var.append(col + "_code")

In [256]:
pca95 = pd.DataFrame(data = pca95)

In [257]:
X = pd.concat([pca95,data[cat_var]], axis = 1)

See Performance on linear model

In [258]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state = 44, test_size = .30)

In [246]:
model = lr.fit(X_train,Y_train)

In [247]:
mean_squared_error(Y_test, model.predict(X_test))

0.02767675437755757

See Performance on Ridge

In [248]:
for i in range (-2, 4):
    alpha = 10**i
    print(alpha,"\n")
    rm = linear_model.Ridge(alpha=alpha)
    ridge_model = rm.fit(X_train, Y_train)
    preds_ridge = ridge_model.predict(X_test)
    print(mean_squared_error(Y_test,preds_ridge))

0.01 

0.027649526701704084
0.1 

0.027416230407243305
1 

0.0258727010128002
10 

0.023516577893032492
100 

0.023058815140612803
1000 

0.0259915665193382


In [268]:
alpha = 100
rm = linear_model.Ridge(alpha=alpha)
ridge_model = rm.fit(X_train, Y_train)
pred = ridge_model.predict(X_test)
print(mean_squared_error(Y_test,pred))

0.022261626543081607


# Ridge Seems to perform well, Generate predictions for test

In [263]:
test_data = pd.DataFrame(data=test_data)

In [264]:
test_data = pd.concat([test_data,test[cat_var]], axis = 1)

In [269]:
pred = ridge_model.predict(test_data)

In [237]:
pred[:6]

array([11.79506994, 12.45020127, 12.09007304, 11.93001105, 12.23363434,
       12.10066322])

In [270]:
pred = np.exp(pred)
submission['SalePrice'] = pred
submission.to_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\submission4.csv", index=False)

# Random Forest Regression

In [249]:
from sklearn.ensemble import RandomForestRegressor

In [250]:
rf = RandomForestRegressor(n_estimators= 10, random_state = 43)

In [259]:
model = rf.fit(X_train,Y_train)

# Try Random forest with PCA variables

In [260]:
mean_squared_error(Y_test, model.predict(X_test))

0.023376573174683727

In [271]:
pred = model.predict(test_data)
pred = np.exp(pred)
submission['SalePrice'] = pred
submission.to_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\submission5.csv", index=False)

# Random forest with n_estimator = 100

In [276]:
rf = RandomForestRegressor(n_estimators= 100, random_state = 43)

In [277]:
model = rf.fit(X_train,Y_train)

In [278]:
mean_squared_error(Y_test, model.predict(X_test))

0.022072292700573294

In [279]:
pred = model.predict(test_data)
pred = np.exp(pred)
submission['SalePrice'] = pred
submission.to_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\submission6.csv", index=False)

In [282]:
print(rf.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 43, 'verbose': 0, 'warm_start': False}


# Trying the above RF model without PCA

In [294]:
#cat.remove("Id")
X = data[cat]

In [295]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state = 44, test_size = .30)

In [296]:
rf = RandomForestRegressor(n_estimators= 100, random_state = 43)
model = rf.fit(X_train,Y_train)
mean_squared_error(Y_test, model.predict(X_test))

0.023670905665964363

# Hyperparameter Tuning on Random Forest - Trying Randomized search

In [304]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest 
n_estimators = [int(x) for x in np.linspace(start = 100, stop =2000, num= 10)]
# Maximum levels in tree(pruning)
max_depth = [int(x) for x in np.linspace(30, 120, num = 10)]
#include None in max_depth
max_depth.append(None)
#Features to consider at each split
max_features = ['auto','sqrt']
#Mininmum samples required to split node
min_samples_split = [2,3,5,10]
# minimum sample required at each leaf node
min_samples_leaf = [1,2,4]
# bootstrap
bootstrap = ['True', 'False']

random_grid = {'n_estimators':n_estimators,
               'max_depth': max_depth,
               'max_features': max_features,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap
               
              }

print(random_grid)


{'n_estimators': [100, 311, 522, 733, 944, 1155, 1366, 1577, 1788, 2000], 'max_depth': [30, 40, 50, 60, 70, 80, 90, 100, 110, 120, None], 'max_features': ['auto', 'sqrt'], 'min_samples_split': [2, 3, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': ['True', 'False']}


In [305]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [306]:
rf_random.fit(X_train,Y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 13.7min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': ['True', 'False'],
                                        'max_depth': [30, 40, 50, 60, 70, 80,
                                                      90, 100, 110, 120, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 3, 5, 10],
                                        'n_estimators': [100, 311, 522, 733,
                                                         944, 1155, 1366, 1577,
                                                         1788, 2000]},
                   random_state=42, verbose=2)

In [307]:
rf_random.best_params_

{'bootstrap': 'False',
 'max_depth': 30,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 944}

In [309]:
mean_squared_error(Y_test, rf_random.best_estimator_.predict(X_test))

0.02260222862314679

In [310]:
test_data = test[cat]

In [314]:
pred = rf_random.best_estimator_.predict(test_data)
pred = np.exp(pred)
submission['SalePrice'] = pred
submission.to_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\submission7.csv", index=False)

# Using the PCA data with rf_random

In [326]:
#training data
pca_data = data[numeric]
pca_data = pca_data.interpolate().dropna()
sc = StandardScaler()
pca_data = sc.fit_transform(pca_data)
pca = PCA(n_components= 0.99)
pca99 = pca.fit_transform(pca_data)
pca99 = pd.DataFrame(data = pca99)
X = pd.concat([pca99,data[cat_var]], axis = 1)
X_train,X_test,Y_train,Y_test = train_test_split(pca99,Y,random_state = 44, test_size = .30)


In [318]:
rf_random.fit(X_train,Y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 20.2min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': ['True', 'False'],
                                        'max_depth': [30, 40, 50, 60, 70, 80,
                                                      90, 100, 110, 120, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 3, 5, 10],
                                        'n_estimators': [100, 311, 522, 733,
                                                         944, 1155, 1366, 1577,
                                                         1788, 2000]},
                   random_state=42, verbose=2)

In [319]:
rf_random.best_params_

{'bootstrap': 'False',
 'max_depth': 100,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'n_estimators': 311}

In [320]:
mean_squared_error(Y_test, rf_random.best_estimator_.predict(X_test))

0.02137561721713638

In [322]:
#test Data
test_data = test[numeric]
test_data = test_data.interpolate().dropna()
test_data = sc.fit_transform(test_data)
test_data = pca.fit_transform(test_data)
test_data = pd.DataFrame(data = test_data)
test_data = pd.concat([test_data, test[cat_var]], axis = 1)
pred = rf_random.best_estimator_.predict(test_data)
pred = np.exp(pred)
submission['SalePrice'] = pred
submission.to_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\submission8.csv", index=False)


In [341]:
data.corr().SalePrice.sort_values()

ExterQual_code      -0.636884
KitchenQual_code    -0.589189
BsmtQual_code       -0.438881
HeatingQC_code      -0.400178
GarageFinish_code   -0.292483
LotShape_code       -0.255580
GarageType_code     -0.223819
BsmtExposure_code   -0.193079
Fence_code          -0.181911
MSZoning_code       -0.166872
KitchenAbvGr        -0.135907
EnclosedPorch       -0.128578
Heating_code        -0.098812
Alley_code          -0.092607
BldgType_code       -0.085591
MSSubClass          -0.084284
OverallCond         -0.077856
MiscFeature_code    -0.069317
LotConfig_code      -0.067396
SaleType_code       -0.054911
YrSold              -0.028923
LowQualFinSF        -0.025606
Id                  -0.021917
MiscVal             -0.021190
BsmtHalfBath        -0.016844
Utilities_code      -0.014314
BsmtFinType1_code   -0.013233
BsmtFinSF2          -0.011378
MasVnrType_code     -0.000488
Condition2_code      0.007513
                       ...   
RoofStyle_code       0.222405
BsmtFullBath         0.227122
PavedDrive

# Using only numeric variables  with PCA

In [327]:
#training data
pca_data = data[numeric]
pca_data = pca_data.interpolate().dropna()
sc = StandardScaler()
pca_data = sc.fit_transform(pca_data)
pca = PCA(n_components= 0.99)
pca99 = pca.fit_transform(pca_data)
pca99 = pd.DataFrame(data = pca99)
#X = pd.concat([pca99,data[cat_var]], axis = 1)
X_train,X_test,Y_train,Y_test = train_test_split(pca99,Y,random_state = 44, test_size = .30)


In [328]:
rf_random.fit(X_train,Y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 16.8min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': ['True', 'False'],
                                        'max_depth': [30, 40, 50, 60, 70, 80,
                                                      90, 100, 110, 120, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 3, 5, 10],
                                        'n_estimators': [100, 311, 522, 733,
                                                         944, 1155, 1366, 1577,
                                                         1788, 2000]},
                   random_state=42, verbose=2)

In [329]:
rf_random.best_params_

{'bootstrap': 'True',
 'max_depth': 120,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 3,
 'n_estimators': 944}

In [330]:
mean_squared_error(Y_test, rf_random.best_estimator_.predict(X_test))

0.022108865498284357

In [331]:
#test Data
test_data = test[numeric]
test_data = test_data.interpolate().dropna()
test_data = sc.fit_transform(test_data)
test_data = pca.fit_transform(test_data)
test_data = pd.DataFrame(data = test_data)
#test_data = pd.concat([test_data, test[cat_var]], axis = 1)
pred = rf_random.best_estimator_.predict(test_data)
pred = np.exp(pred)
submission['SalePrice'] = pred
submission.to_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\submission9.csv", index=False)


# Trying different models

In [349]:
#training data
X = data[cat]
print(X.shape)
X_train,X_test,Y_train,Y_test = train_test_split(pca99,Y,random_state = 44, test_size = .30)
#X_train.drop(['Condition2_code'],axis =1)


(1460, 79)


In [350]:
rf_random.fit(X_train,Y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 16.0min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': ['True', 'False'],
                                        'max_depth': [30, 40, 50, 60, 70, 80,
                                                      90, 100, 110, 120, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 3, 5, 10],
                                        'n_estimators': [100, 311, 522, 733,
                                                         944, 1155, 1366, 1577,
                                                         1788, 2000]},
                   random_state=42, verbose=2)

In [351]:
rf_random.best_params_

{'bootstrap': 'False',
 'max_depth': 80,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 3,
 'n_estimators': 100}

In [352]:
mean_squared_error(Y_test,rf_random.best_estimator_.predict(X_test))

0.022211626817796776

In [ ]:
#test Data
test_data = test[numeric]
test_data = test_data.interpolate().dropna()
test_data = sc.fit_transform(test_data)
test_data = pca.fit_transform(test_data)
test_data = pd.DataFrame(data = test_data)
test_data = pd.concat([test_data, test[cat_var]], axis = 1)
pred = rf_random.best_estimator_.predict(test_data)
pred = np.exp(pred)
submission['SalePrice'] = pred
submission.to_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\submission8.csv", index=False)


In [435]:
data = pd.read_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\train.csv")
test = pd.read_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\test.csv")
data.shape

(1460, 81)

In [442]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=10, random_state=0)
train = data.drop(["Id"],axis = 1)
test_data = test.drop(["Id"],axis = 1)
train = imp.fit_transform(data[numeric])
pca = PCA(n_components= 0.99)
train = pca.fit_transform(train)
test_data = imp.fit_transform(test_data[numeric])

In [443]:
for cols in cat_cols.columns:
    data[cols + "_code" ] = data[cols].astype('category').cat.codes

In [444]:
train = pd.DataFrame(data= train)
train = pd.concat([train, data[cat_var]], axis = 1)

In [445]:
X_train,X_test,Y_train,Y_test = train_test_split(train,Y,random_state = 44, test_size = .30)

In [446]:
rf = RandomForestRegressor(n_estimators=100, random_state=43)
model = rf.fit(X_train,Y_train)
mean_squared_error(Y_test,model.predict(X_test))

0.026556904446667598

In [426]:
from sklearn.impute import SimpleImputer

cat_data = data.select_dtypes(exclude=[np.number])
cat_data = cat_data.drop(['PoolQC', 'MiscFeature','Alley', 'Fence', 'FireplaceQu'], axis =1)
imp = SimpleImputer(strategy="most_frequent")
cat_data = imp.fit_transform(cat_data)
cat_data = pd.DataFrame(data= cat_data)


In [427]:
c_df = pd.DataFrame()
for cols in cat_data.columns:
    c_df[str(cols) + "_code" ] = cat_data[cols].astype('category').cat.codes

for cols in ['PoolQC', 'MiscFeature','Alley', 'Fence', 'FireplaceQu']:
    c_df[str(cols) + "_code" ] = data[cols].astype('category').cat.codes

In [437]:
train = pd.DataFrame(data = train)
train = pd.concat([train,c_df], axis =1)


In [429]:
train.shape

(1460, 79)

In [430]:
X_train,X_test,Y_train,Y_test = train_test_split(train,Y,random_state = 44, test_size = .30)

In [439]:
rf = RandomForestRegressor(n_estimators=100, random_state=43)
model = rf.fit(X_train,Y_train)
mean_squared_error(Y_test,model.predict(X_test))

0.026539539160579093

# XGBoost

In [487]:
data = pd.read_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\train.csv")
test = pd.read_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\test.csv")
data.shape
test.shape

(1459, 80)

In [527]:
cat_cols = data.select_dtypes(exclude=[np.number])
for cols in cat_cols.columns:
    data[cols + "_code" ] = data[cols].astype('category').cat.codes    
    
data = data.select_dtypes(include = [np.number]).interpolate().dropna()
train = data.drop(["Id","SalePrice"],axis = 1)

train = data[numeric]
train = sc.fit_transform(train)

pca = PCA(n_components= 0.99)
train = pca.fit_transform(train)
train = pd.DataFrame(train)
train = pd.concat([train,data[cat_var]],axis = 1)

Y = np.log(data.SalePrice)

X_train,X_test,Y_train,Y_test = train_test_split(train,Y,random_state = 44, test_size = .30)

from xgboost import XGBRegressor
model = XGBRegressor(objective='reg:squarederror')
model.fit(X_train,Y_train)
mean_squared_error(Y_test,model.predict(X_test))

0.019410647022091967

In [542]:
for cols in cat_cols.columns:
    test[cols + "_code" ] = test[cols].astype('category').cat.codes  
test = test.select_dtypes(include = [np.number]).interpolate().dropna()
test_data = test[numeric]
test_data = sc.fit_transform(test_data)
test_data = pca.fit_transform(test_data)
test_data = pd.DataFrame(data = test_data)
test_data = pd.concat([test_data,test[cat_var]],axis = 1)


(1459, 75)

In [543]:
#submiission = 
pred = model.predict(test_data)
pred = np.exp(pred)
submission['SalePrice'] = pred
submission.to_csv(r"C:\Users\navee\Downloads\ML Materials\house-prices-advanced-regression-techniques\submission10.csv", index=False)
